In [ ]:
import pandas as pd
import numpy as np
pd.set_option('max_columns',None)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox, yeojohnson, skew, kurtosis
from imblearn.over_sampling import SVMSMOTE
import gc
from tqdm import tqdm
import os
from keras import layers
np.errstate(divide='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import keras
from keras.layers import Input, Dense, BatchNormalization, Concatenate, Dropout
from tensorflow.keras.utils import to_categorical
import math
import keras.backend as K
from tensorflow.keras.optimizers import Nadam, Adam

from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Input, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Multiply

In [ ]:
debug = False
plot = False

train = pd.read_csv('../input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-dec-2021/test.csv')


# plabel = pd.read_csv('../input/tps12-pseudolabels/tps12-pseudolabels_v2.csv')
# df = pd.read_csv('../input/bts-12-external-data/covtype.data',delimiter=',')

# df.columns = train.columns.tolist()
# train = pd.concat([train,plabel],axis=0)

train.drop(['Id'], axis=1, inplace=True)

train = train[train['Cover_Type'] !=5].reset_index(drop=True)


if debug:
    train = train.sample(10000)
    test = test.sample(10000)


y = train['Cover_Type']
train.drop(['Cover_Type'], axis=1, inplace=True)
test.drop(['Id'], axis=1, inplace=True)

In [ ]:
#reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
to_drop = ['Soil_Type15','Soil_Type7','Soil_Type1']

train.drop(to_drop, inplace=True, axis=1)
test.drop(to_drop, inplace=True, axis=1)

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = pd.DataFrame(encoder.fit_transform(y.values.reshape(-1,1)), columns=['Cover_Type'])

# Preprocessing

![](https://www.photopills.com/sites/default/files/tutorials/2014/2-azimuth-elevation.jpg)

In [ ]:
horizontal_cols = ['Horizontal_Distance_To_Hydrology','Horizontal_Distance_To_Roadways','Horizontal_Distance_To_Fire_Points']
hillshade_cols = ['Hillshade_9am','Hillshade_Noon','Hillshade_3pm']
wilderness_cols = ['Wilderness_Area1','Wilderness_Area2','Wilderness_Area3','Wilderness_Area4']
Soil_cols = ['Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11',
       'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']

In [ ]:
if plot:
    for col in horizontal_cols:
        sns.kdeplot(data=train,x=col, fill=True, color='red',alpha=0.5)
        sns.kdeplot(data=test,x=col, fill=True, color = 'blue',alpha=0.5)
        plt.show()

In [ ]:
if plot:
    for col in hillshade_cols:
        sns.kdeplot(data=train,x=col, fill=True, color='red',alpha=0.5)
        sns.kdeplot(data=test,x=col, fill=True, color = 'blue',alpha=0.5)
        plt.show()

In [ ]:
display(train.head(4))
test.head(4)

0 - Spruce/Fir

1 - Lodgepole Pine

2 - Ponderosa Pine

3 - Cottonwood/Willow

4 - Douglas-fir

5 - Krummholz

In [ ]:
train.isnull().sum().sum()

![](https://miro.medium.com/max/814/0*_9ljPf7RbVI5cVdG.png)

In [ ]:
def safe_log10(x, eps=1e-10):
    result = np.where(x > eps, x, -10)
    np.log10(result, out=result, where=result > 0)
    return result

In [ ]:
#['original','log1p','sqrt','crbt','boxcox','yeojohnson']
from numpy import log1p

to_trans = {'Elevation':'yeojohnson','Aspect':'yeojohnson','Slope':'yeojohnson','Horizontal_Distance_To_Hydrology':'yeojohnson',
            'Vertical_Distance_To_Hydrology':'yeojohnson','Horizontal_Distance_To_Roadways':'yeojohnson','Hillshade_9am':'yeojohnson',
           'Hillshade_Noon':'yeojohnson','Hillshade_3pm':'yeojohnson','Horizontal_Distance_To_Fire_Points':'yeojohnson'}

to_trans_df = pd.DataFrame.from_dict({'keys':to_trans.keys(), 'values':to_trans.values()})

for i in to_trans_df['keys']:
    train[i] = locals()[to_trans_df[to_trans_df['keys'] == i]['values'].values[0]](train[i])[0]
    test[i] = locals()[to_trans_df[to_trans_df['keys'] == i]['values'].values[0]](test[i])[0]
    print(i,'Done!')

In [ ]:
#adding new columns and iterating over model to check the performance
def euclidean(a,b):
    dis = np.sqrt((a.astype('int64')**2) + (b.astype('int64')**2))
    
    return dis

def aspect(x):
    if x>359:
        return x-360
    elif x<0:
        return x+360
    else:
        return x

def manhattan(a,b):
    dis = abs(a.astype('int64')) + abs(b.astype('int64'))
    
    return dis

#================================= forest to Hydrology Euclidean ===================================================
train['ForestHydrology_euclidean'] = euclidean(train['Horizontal_Distance_To_Hydrology'],train['Vertical_Distance_To_Hydrology'])
test['ForestHydrology_euclidean'] = euclidean(test['Horizontal_Distance_To_Hydrology'],test['Vertical_Distance_To_Hydrology'])

#================================= forest to Hydrology manhattan ===================================================
train['ForestHydrology_manhattan'] = manhattan(train['Horizontal_Distance_To_Hydrology'],train['Vertical_Distance_To_Hydrology'])
test['ForestHydrology_manhattan'] = manhattan(test['Horizontal_Distance_To_Hydrology'],test['Vertical_Distance_To_Hydrology'])

#================================= Roadway to Hydrology pytha ===================================================
train['RoadwayHydrology_pytha'] = euclidean(train['Horizontal_Distance_To_Hydrology'],train['Horizontal_Distance_To_Roadways'])
test['RoadwayHydrology_pytha'] = euclidean(test['Horizontal_Distance_To_Hydrology'],test['Horizontal_Distance_To_Roadways'])

#================================= Roadway to Fire_Points pytha ===================================================
train['RoadwayFirepoints_pytha'] = euclidean(train['Horizontal_Distance_To_Fire_Points'],train['Horizontal_Distance_To_Roadways'])
test['RoadwayFirepoints_pytha'] = euclidean(test['Horizontal_Distance_To_Fire_Points'],test['Horizontal_Distance_To_Roadways'])

#================================= Hydrology to Fire Pytha pytha ===================================================
train['FirepointsHydrology_pytha'] = euclidean(train['Horizontal_Distance_To_Hydrology'],train['Horizontal_Distance_To_Fire_Points'])
test['FirepointsHydrology_pytha'] = euclidean(test['Horizontal_Distance_To_Hydrology'],test['Horizontal_Distance_To_Fire_Points'])


train['Horizontal_sum'] = train[horizontal_cols].astype('int64').sum(axis=1)
test['Horizontal_sum'] = test[horizontal_cols].astype('int64').sum(axis=1)

train['Aspect'] = train['Aspect'].apply(aspect)
test['Aspect'] = test['Aspect'].apply(aspect)

eu = ['ForestHydrology_euclidean','RoadwayHydrology_pytha','RoadwayFirepoints_pytha','FirepointsHydrology_pytha']

train['Horizontal_min'] = train[horizontal_cols].min(axis=1)
train['Horizontal_mean'] = train[horizontal_cols].mean(axis=1)
train['Horizontal_max'] = train[horizontal_cols].max(axis=1)

test['Horizontal_min'] = test[horizontal_cols].min(axis=1)
test['Horizontal_mean'] = test[horizontal_cols].mean(axis=1)
test['Horizontal_max'] = test[horizontal_cols].max(axis=1)

train['eu_min'] = train[eu].min(axis=1)
train['eu_mean'] = train[eu].mean(axis=1)
train['eu_max'] = train[eu].max(axis=1)

test['eu_min'] = test[eu].min(axis=1)
test['eu_mean'] = test[eu].mean(axis=1)
test['eu_max'] = test[eu].max(axis=1)

In [ ]:
train.shape, y.shape

In [ ]:
def fe(df):
    ### source: https://www.kaggle.com/c/tabular-playground-series-dec-2021/discussion/293373
    df.loc[df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
    df.loc[df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
    df.loc[df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
    df.loc[df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
    df.loc[df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
    df.loc[df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
    ########
    return df

train = fe(train)
test = fe(test)

In [ ]:
train.isnull().sum().sum(), train.shape, test.shape, y.shape

In [ ]:
to_ignore = ('id','target','Cover_Type','Wilderness_Area','Soil_Type')

cat_cols = [col for col in train.columns if not col.startswith(to_ignore)]

train['wilderness_encoder'] = np.argmax(train[wilderness_cols].values,axis=1)
test['wilderness_encoder'] = np.argmax(test[wilderness_cols].values,axis=1)

train['soil_encoder'] = np.argmax(train[Soil_cols].values,axis=1)
test['soil_encoder'] = np.argmax(test[Soil_cols].values,axis=1)

cat_cols.append('soil_encoder')

In [ ]:
from sklearn.preprocessing import LabelEncoder

test['Cover_Type'] = -1

cat_data = pd.concat([train.join(y),test],axis=0)

for feat in tqdm(cat_data[cat_cols].columns):
    encoder = LabelEncoder()
    cat_data.loc[:,feat] = encoder.fit_transform(cat_data[feat].astype('str').fillna('-1').values)


In [ ]:
def get_emb_model(df, categorical_columns):
    inputs = []
    outputs = []
    
    for c in categorical_columns:
        num_unique_vals = int(df[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_vals / 2), 50))
        inp = Input(shape = (1,))
        out = layers.Embedding(num_unique_vals+1, embed_dim, name=c)(inp)
        out = Dropout(0.3)(out)
        out = BatchNormalization()(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x = Concatenate()(outputs)
    x = Dense(300, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    
    model = Model(inputs=inputs, outputs=x)
    return model

In [ ]:
model = get_emb_model(cat_data,cat_cols)

In [ ]:
model.summary()

In [ ]:
cat_data.reset_index(drop=True).to_feather('cat_train_test.feather')
X.join(y).reset_index(drop=True).to_feather('train.feather')
test.reset_index(drop=True).to_feather('test.feather')

In [ ]:
break

# Modeling

In [ ]:
X = train.copy()
classes = y.nunique().values[0]

del train
gc.collect()

In [ ]:
def plot_transformer(df,column,rows=2,cols=3,only_og=False):
    from scipy.stats import boxcox, yeojohnson, skew, kurtosis

    #setting plot theme
    plt.rcParams['figure.dpi'] = 300

    fig = plt.figure(figsize=(9,6), facecolor='#f6f5f5', constrained_layout='tight_layout')
    gs = fig.add_gridspec(rows, cols)

    background_color = "#f6f5f5"
    sns.set_palette(['#ff355d','#ffd514',])

    #making multiple ax
    ax_dict = {}
    for row in range(rows):
        for col in range(cols):
            ax_dict["ax%s%s" %(row,col)] = fig.add_subplot(gs[row, col])
    
    if np.any(df[column] <= 0):
        df_scaled = (df[column] - df[column].min(axis=0)) / (df[column].max(axis=0) - df[column].min(axis=0))
        df_scaled = df_scaled * (1 - 0.001) + 0.001
    else:
        df_scaled = df[column]
        
    #dictionary of data transformation
    pp = {0: df[column],1:np.log1p(df_scaled),2:np.sqrt(df[column]),3:np.cbrt(df[column])
          ,4:boxcox(df_scaled)[0],5:yeojohnson(df[column])[0]}

    #iterating over subplots and ploting each one by one.        
    locals().update(ax_dict)
    count = 0

    trans = ['original','log1p','sqrt','crbt','boxcox','yeojohnson']
    #setting theme for every ax in local()
    for indx,row in enumerate(range(rows)):
        for col in range(cols):

            locals()['ax' + str(row) + str(col)].tick_params(labelsize=3, width=0.5, length=1.5)

            #comment below 2 lines if you are using regplot
            locals()['ax' + str(row) + str(col)].grid(which='major', axis='x', zorder=0, color='#EEEEEE', linewidth=0.4)
            locals()['ax' + str(row) + str(col)].grid(which='major', axis='y', zorder=0, color='#EEEEEE', linewidth=0.4)

            for s in ["right", "top"]:
                locals()['ax' + str(row) + str(col)].spines[s].set_visible(False)


            locals()['ax' + str(row) + str(col)].set_facecolor(background_color)

            ax_sns = sns.kdeplot(x=pp[count],ax=locals()['ax' + str(row) + str(col)],zorder=4,fill=True,ec='black')

            locals()['ax' + str(row) + str(col)].set_facecolor(background_color)
            locals()['ax' + str(row) + str(col)].set_xlabel(column,fontsize=4, weight='bold',)
            locals()['ax' + str(row) + str(col)].set_ylabel(trans[count],fontsize=4, weight='bold')
            
            locals()['ax' + str(row) + str(col)].title.set_text(f'skew: {round(skew(pp[count]),2)} kurtosis: {round(kurtosis(pp[count]),2)}')
            if only_og:break
            count+=1

    plt.show()

In [ ]:
from keras import Input, layers, Model
from keras.layers import Dense

def get_model(SEED=42,df=None, categorical_columns=None):
    #for reuseability
    keras.backend.clear_session()
    tf.keras.backend.clear_session()
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)
    
    #input
    input_ = Input(shape=(X.shape[1]))
    
    #layers
    x1 = Dense(units=384, activation='swish')(input_)
    x1 = Dropout(rate=0.45)(x1)
    x1 = BatchNormalization()(x1)
    
    
    x2 = Dense(units=192, activation='swish')(x1)
    x2 = Dropout(rate=0.35)(x2)
    x2 = BatchNormalization()(x2)
    
    x3 = Dense(units=96, activation='swish')(x2)
    x3 = Dropout(rate=0.25)(x3)
    x3 = BatchNormalization()(x3)
    
    x4 = Dense(units=192, activation='swish')(x3)
    x4 = BatchNormalization()(x4)
    x4 = Multiply()([x2, x4])
    x4 = Dropout(rate=0.35)(x4)
    x4 = BatchNormalization()(x4)
    
    x5 = Dense(units=384, activation='swish')(x4)
    x5 = BatchNormalization()(x5)
    x5 = Multiply()([x1, x5])
    x5 = Dropout(rate=0.45)(x5)
    x5 = BatchNormalization()(x5)
    
    x = Concatenate()([x3, x5])
    x = Dense(units=128, activation='swish')(x)
    x = Dropout(rate=0.25)(x)
    x = BatchNormalization()(x)
    
    inputs = []
    outputs = []
    
    for c in categorical_columns:
        num_unique_vals = int(df[c].nunique())
        embed_dim = int(min(np.ceil(num_unique_vals / 2), 50))
        inp = Input(shape = (1,))
        out = layers.Embedding(num_unique_vals+1, embed_dim, name=c)(inp)
        out = Dropout(0.3)(out)
        out = BatchNormalization()(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
    
    x1 = Concatenate()(outputs)
    x1 = Dense(300, activation='relu')(x1)
    x1 = Dropout(0.3)(x1)
    x1 = BatchNormalization()(x1)
    
    x = Concatenate()([x,x1])
    
    x = Dense(units=64, activation='swish')(x)
    x = Dropout(rate=0.15)(x)
    x = BatchNormalization()(x)
    
    x = Dense(units=32, activation='swish')(x)
    x = Dropout(rate=0.05)(x)
    x = BatchNormalization()(x)
    

    #output
    output_ = Dense(classes, activation='softmax')(x)
    
    model = Model(inputs=[input_,inputs], outputs =[output_])
    
    return model

In [ ]:
model = get_model(42,cat_data, cat_cols)

In [ ]:
tf.keras.utils.plot_model(model,show_dtype=True, show_layer_names=True)

In [ ]:
K = keras.backend

class ExponentialLearningRate(keras.callbacks.Callback):
    def __init__(self, factor):
        self.factor = factor
        self.rates = []
        self.losses = []
    def on_epoch_begin(self, epoch, logs=None):
        self.prev_loss = 0
    def on_batch_end(self, batch, logs=None):
        batch_loss = logs["loss"] * (batch + 1) - self.prev_loss * batch
        self.prev_loss = logs["loss"]
        self.rates.append(K.get_value(self.model.optimizer.learning_rate))
        self.losses.append(batch_loss)
        K.set_value(self.model.optimizer.learning_rate, self.model.optimizer.learning_rate * self.factor)
        
def find_learning_rate(model, X, y, epochs=1, batch_size=32, min_rate=10**-5, max_rate=10):
    init_weights = model.get_weights()
    iterations = math.ceil(len(X) / batch_size) * epochs
    factor = np.exp(np.log(max_rate / min_rate) / iterations)
    init_lr = K.get_value(model.optimizer.learning_rate)
    K.set_value(model.optimizer.learning_rate, min_rate)
    exp_lr = ExponentialLearningRate(factor)
    history = model.fit(X, y, epochs=epochs, batch_size=batch_size,
                        callbacks=[exp_lr])
    K.set_value(model.optimizer.learning_rate, init_lr)
    model.set_weights(init_weights)
    return exp_lr.rates, exp_lr.losses

def plot_lr_vs_loss(rates, losses):
    plt.plot(rates, losses)
    plt.gca().set_xscale('log')
    plt.hlines(min(losses), min(rates), max(rates))
    plt.axis([min(rates), max(rates), min(losses), (losses[0] + min(losses)) / 2])
    plt.xlabel("Learning rate")
    plt.ylabel("Loss")

In [ ]:
X.isnull().sum().sum()

In [ ]:
X.shape, y.shape, test.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler

kf = StratifiedKFold(n_splits=10,shuffle=True,random_state=1)

scores = []
oof_pred = {}
test_pred = {}

model_name = 'DNN'
cce = tf.keras.metrics.Accuracy()

oof_pred = dict()
test_pred = dict()
scores = dict()

indx = []

model_name = 'DNN'

oof_pred[model_name] = list()
oof_pred['y_valid'] = list()
test_pred[model_name] = list()
scores[model_name] = list()

for fold,(train_indx, valid_indx) in enumerate(kf.split(X,y)):
    print(f"{'='*10} Fold: {fold} {'='*10}")
    
    #==================== data spliting ====================
    X_train, X_valid = X.iloc[train_indx],X.iloc[valid_indx]
    y_train, y_valid = y.iloc[train_indx],y.iloc[valid_indx]
    #==================== preprocesing ====================
    scaler = RobustScaler()
    
    X_train = scaler.fit_transform(X_train)
    
    X_valid = scaler.transform(X_valid)
    test_scaled = scaler.transform(test)
    
    #==================== NOTES: Additional steps ====================
    

    #====================get model, callbacks and compile ====================
    
    model = get_model(SEED=42)
    
    kall1 = keras.callbacks.EarlyStopping(monitor="val_loss",
    patience=20,
    mode="auto",
    restore_best_weights=True)
    
    kall2 = tf.keras.callbacks.LearningRateScheduler(tf.keras.optimizers.schedules.ExponentialDecay(
    1e-2,decay_steps=X_train.shape[0]//1024,
    decay_rate=0.96,
    staircase=True
    ))
    
    model.compile(optimizer=Nadam(learning_rate = 1e-2),
    loss='sparse_categorical_crossentropy',
    metrics='acc')
    
    #==================== OneCycleScheduler ====================
    
#     batch_size = 1024
#     rates, losses = find_learning_rate(model, X_train, y_train, epochs=1, batch_size=batch_size)
#     plot_lr_vs_loss(rates, losses)
#     plt.show()
#     break
    #==================== model ====================
    
    history = model.fit(X_train,y_train,epochs=5000, batch_size=3024, validation_data = (X_valid, y_valid), callbacks = [kall1,kall2],
             verbose=0)
    
    #==================== Loss Plot ====================
    lol = pd.DataFrame(history.history['loss']).plot(label='loss')
    pd.DataFrame(history.history['val_loss']).plot(ax=lol, label='val_loss')
    
    plt.legend(['loss','val_loss'])

    plt.show()
    
    print()
    
    
    #==================== Model Prediction ====================
    
    y_pred = model.predict(X_valid)
    oof_pred[model_name].extend(np.argmax(y_pred,axis=-1))
    
    score = cce(y_valid, np.argmax(y_pred,axis=-1)).numpy()
    scores[model_name].append(score)
    
    y_pred = model.predict(test_scaled)
    test_pred[model_name].append(np.argmax(y_pred,axis=-1)) #appending test prediction for every fold
    
    oof_pred['y_valid'].extend(y_valid.iloc[:,-1].values)
    
    #==================== printing fold number and score ====================
    print(f"Fold {fold} Score: {score}")
    
    #==================== deleting some data and clearning ====================
    del model, X_valid, X_train, y_valid, y_train, y_pred
    gc.collect()
    
print('\n','=-='*20,f'Overall Score: {np.mean(scores[model_name])}','=-='*20)

In [ ]:
from scipy import stats
name = 'DNN'

try:
    sample_submission.head(1)
except:
    sample_submission = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')


base_test_predictions = pd.DataFrame(
    {name: np.mean(np.column_stack(test_pred[name]), axis=1)
    for name in test_pred.keys()}
)

def decode(x):
    if x<4:
        return x+1
    else:
        return x+2


base_submission = sample_submission.copy()

base_submission.iloc[:,-1] = base_test_predictions.iloc[:,-1].apply(decode).astype('int8')
base_submission.to_csv(f'{name}_base.csv',index=False)

oof_predictions = pd.DataFrame({name:oof_pred[name] for name in oof_pred.keys()})
oof_predictions.loc[:,name] = oof_predictions.loc[:,name].apply(decode).astype('int8') 
oof_predictions.to_csv(f'{name}_oof_pred.csv',index=False)